In [81]:
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque
from math import gcd, ceil
import re
import networkx as nx
from dataclasses import dataclass
import itertools
import aoc_utils
from matplotlib import pyplot as plt
import string
# plt.imshow(pic)
from itertools import product

In [3]:
f = open('test.txt').read().split('\n\n')
rules, messages = [part.split('\n') for part in f]

rulebook = {}
for r in rules:
    num, ins = r.split(': ')
    ins = ins.replace('"','')
    ins_piped = tuple(tuple(i.split()) for i in ins.split('|'))
    if len(ins_piped[0])==1 and ins_piped[0][0] in 'ab':
        ins_piped = ins_piped[0]
    rulebook[num]=ins_piped

In [2]:
def test(s,seq):
    if s == '' or seq == []:
        return s == '' and seq == [] # if both are empty, True. If only one, False.
    
    r = rules[seq[0]]
    if '"' in r:
        if s[0] in r:
            return test(s[1:], seq[1:]) # strip first character
        else:
            return False # wrong first character
    else:
        return any(test(s, t + seq[1:]) for t in r) # expand first term

def parse_rule(s):
    n,e = s.split(": ")
    if '"' not in e:
        e = [[int(r) for r in t.split()] for t in e.split("|")]
    return (int(n),e)

rule_text, messages = [x.splitlines() for x in open("input.txt").read().split("\n\n")]
rules = dict(parse_rule(s) for s in rule_text)            
print("Part 1:", sum(test(m,[0]) for m in messages))       

rule_text += ["8: 42 | 42 8","11: 42 31 | 42 11 31"]
rules = dict(parse_rule(s) for s in rule_text)
print("Part 2:", sum(test(m,[0]) for m in messages)) 

Part 1: 129
Part 2: 243


In [182]:
class CFG():
    """"Takes a grammer as dict with tuple of options as values. Terminal values should not be in a tuple but as a string
    Usage:
        cfg = CFG(grammar_dict)
            reverse as optional parameter when k,v are reversed
        cfg converts the grammar to Choemsky Normal form by taking care of options and unit productions
    """
    def __init__(self, grammar = None, reverse = True):
        self.outcomes = defaultdict(set)
        if grammar:
            # convert grammar to CNF and add terminals to outcomes
            self.grammar = self.grammar_to_cnf(grammar, reverse)
            self.outcomes.update({k:v for k,v in self.grammar.items() if isinstance(k, str)})


    def grammar_to_cnf(self, grammar, reverse):
        grammar = self.remove_options(grammar, reverse)
        grammar = self.remove_triplets(grammar)
        return self.remove_unit_productions(grammar)

    def remove_triplets(self, grammar):
        new_grammar = defaultdict(set)
        for k,v in grammar.items():
            if len(k) > 2:
                for i, t in enumerate(k[0:-2]):
                    print(i,t)
                    newvar = str(v) + '_' + str(i)
                    oldvar = str(v) + '_' + str(i-1)
                    if i == 0:
                        new_grammar[t,newvar] = v
                    else:
                        new_grammar[t,newvar] = {oldvar}
                new_grammar[k[-2:]].add(newvar)
            else:
                new_grammar[k] |= v
        return new_grammar
            

    def remove_options(self, grammar, reverse):
        # step to get to Chomsky Normal Form
        new_grammar = defaultdict(set)
        if reverse:
            for k,v in grammar.items():
                for option in v:
                    new_grammar[option].add(k)
        else:
            for k,v in grammar.items():
                for option in k:
                    new_grammar[option].add(v)
        return new_grammar
        
    def remove_unit_productions(self,grammar):
        # step to get to Choemsky Normal Form
        singulars = {k[0]:next(iter(v)) for k,v in grammar.items() if len(k)!=2 and not isinstance(k,str)}
        for k,v in singulars.items():
            for j in grammar.values():
                if k in j:
                    j.add(v)
        return grammar


    def pieces(self, test,l):
        assert isinstance(test, str)
        # gets all possibilities of len out of a string
        # parts = [test[i:] for i in range(l)]
        return {test[i:i+l] for i in range(len(test)-l+1) if test[i:i+l] not in self.outcomes}
        # return {comb for comb in zip(*parts)}

    def splitter(self,option):
        assert isinstance(option, str)
        # splits list into all options of two
        return {(option[:i], option[i:]) for i in range(1,len(option))}

    def check_possible_option(self, option):
        first = self.outcomes.get(option[0],[])
        second = self.outcomes.get(option[1],[])
        res = set()
        for potential in product(first,second):
            if potential in self.grammar: res |= self.grammar[potential]
        return res

    def solve(self, messages):
        for num, m in enumerate(messages):
            if num % 40 == 0: print(num)
            for i in range(2,len(m)+1):
                for j in self.pieces(m, i):
                    for k in self.splitter(j):
                        res = self.check_possible_option(k)
                        if res:
                            self.outcomes[j] |= res
        return self.outcomes
          
cfg = CFG()
assert cfg.pieces('abcde',3) == {'abc', 'bcd', 'cde'}
assert cfg.splitter('abcd') == {('a', 'bcd'), ('ab', 'cd'), ('abc', 'd')}


In [183]:
cfg = CFG(rulebook)
out = cfg.solve(messages)
sum([1 for m in messages if (m in out) and ('0' in out[m])])

0 4
0


2

In [184]:
rulebook

{'0': (('4', '1', '5'),),
 '1': (('2', '3'), ('3', '2')),
 '2': (('4', '4'), ('5', '5')),
 '3': (('4', '5'), ('5', '4')),
 '4': ('a',),
 '5': ('b',)}

In [108]:
from collections import namedtuple

def test(message, rules, r):
    print('called', message, r)
    if rules[r]=='a' or rules[r]=='b':
        print('endpoint reached',message and message[0] == rules[r])
        return {1,} if (message and message[0] == rules[r]) else set()
    else:
        overall_matches = set()
        for opt in rules[r]:
            opt_match = {0,}
            for rule in opt:
                new_match = set()
                for n in opt_match:
                    new_match |= {n+m for m in test(message[n:],rules,rule)}
                opt_match = new_match
            overall_matches |= opt_match
        print('returning', overall_matches)
        return overall_matches

print("Part 1:", sum(len(m) in test(m,rulebook,'0') for m in messages))

called ababbb 0
called ababbb 4
endpoint reached True
called babbb 1
called babbb 2
called babbb 4
endpoint reached False
called babbb 5
endpoint reached True
called abbb 5
endpoint reached False
returning set()
called babbb 3
called babbb 4
endpoint reached False
called babbb 5
endpoint reached True
called abbb 4
endpoint reached True
returning {2}
called bbb 2
called bbb 4
endpoint reached False
called bbb 5
endpoint reached True
called bb 5
endpoint reached True
returning {2}
returning {4}
called b 5
endpoint reached True
returning {6}
called bababa 0
called bababa 4
endpoint reached False
returning set()
called abbbab 0
called abbbab 4
endpoint reached True
called bbbab 1
called bbbab 2
called bbbab 4
endpoint reached False
called bbbab 5
endpoint reached True
called bbab 5
endpoint reached True
returning {2}
called bab 3
called bab 4
endpoint reached False
called bab 5
endpoint reached True
called ab 4
endpoint reached True
returning {2}
called bbbab 3
called bbbab 4
endpoint reac

In [185]:
messages

['ababbb', 'bababa', 'abbbab', 'aaabbb', 'aaaabbb']

In [59]:
def check_valid(rulebook, rule, inp):
    cur_rule = rulebook[rule]
    if isinstance(cur_rule, string):
        if cur_rule == inp[0]: yield inp[1:]
    else:
        for i in cur rul: yield from check_
    else:
        # len is longer so we have options
        return valid_seq(rulebook, rule, input)


def valid_single():
    

def valid_seq():
    yield from valid

{0: [[4, 1, 5]],
 1: [[2, 3], [3, 2]],
 2: [[4, 4], [5, 5]],
 3: [[4, 5], [5, 4]],
 4: [['a']],
 5: [['b']]}

In [ ]:
def test(message, rules, r):
    if rules[r].val:
        return {1,} if (message and message[0] == rules[r].val) else set()
    else:
        overall_matches = set()
        for opt in rules[r].opts:
            opt_match = {0,}
            for rule in opt:
                new_match = set()
                for n in opt_match:
                    new_match |= {n+m for m in test(message[n:],rules,rule)}
                opt_match = new_match
            overall_matches |= opt_match
        return overall_matches

In [44]:
def isvalid(test, option):
    if len(option[0]) > 1: # multiple options, any of them is good
        if any(isvalid(test, o for o in option):
            return True
    else:
        if len(option[0][0]==1):
            if option[0][0] == test:
                return True
            elif option[0][0] in rulebook:
                yield from isvalid(test, option[0][0])
            else:
                return False
        else:
            for i 

    print(test, option)
    
    return False

a [['a']]


False

In [45]:
test = lines[0]
current_rule = 0
if any(isvalid(test, option) for option in rulebook[current_rule]):
    print('valid')
else:
    print('false')

KeyError: 0

In [63]:
f=open('rules.txt')
f=open('rulestest.txt')
f=open('oldrules.txt')
lines = [line.rstrip('\n') for line in f]
rules = {}
for line in lines:
    num, ins = line.split(': ')
    ins = ins.split('|')
    rules[num] = [i.split() for i in ins]


f=open('messages.txt')
# f=open('messagestest.txt')
mess = [line.rstrip('\n') for line in f]


def getoptions(key):
    ins = rules[key]
    alloptions = []
    for option in ins: # go through all the options, can be 1 if 1 option. this can still yield multiple results through recursion
        # print('option', option)
        ans = []
        for i in option:
            if i.isnumeric():
                res = getoptions(i)
                ans.append(res)
            else:
                ans.append(i) # i is a or b

        temp = [''.join(comb) for comb in itertools.product(*ans)]
            # if len(temp)==1:
            #     alloptions.append(temp[0])
            # else:
        alloptions.append(temp)
    if len(alloptions)==1:
        return alloptions[0]
    else:
        # print(aoc.flatten(alloptions))
        return aoc.flatten(alloptions)


128

In [46]:
ins = getoptions('0')
ans =0
for line in mess:
    if line in ins: ans +=1
ans

129

In [50]:
# f=open('rulestest.txt')
lines = [line.rstrip('\n') for line in f]
rules = {}
for line in lines:
    num, ins = line.split(': ')
    ins = ins.split('|')
    rules[num] = [i.split() for i in ins]
getoptions('0')


KeyError: '0'

In [70]:
forty = getoptions('42')
thirty = getoptions('31')
8 + 11
42 + optioneelzichzelf
11: 42 31 | 42 11 31

42 + (optioneel 42) + 42 (optioneel combo 42 en 31) 31


In [77]:
cut = len(max((m for m in mess), key = len))
combinations = cut/8
combinations
def gen_forty():
    for i in range(1,combinations+1)
    len = 1

    

12.0

In [152]:
m = mess[0]
ans = 0
nok = 0

for m in mess:
    
    cut = len(m)//8
    # print('new ', cut)
    darkmode = False
    nextforty = True
    assert cut * 8 == len(m)
    for i in range(cut):
        # print(i)
        # print(m[i*8:i*8+8])
        temp = m[i*8:i*8+8]
        # print(len(temp))
        if (i == 0 and temp not in forty) or (i == 1 and temp not in forty):
            # print('begin nok')
            nok+=1
            break
        elif i > 1:
            if (i == cut-1):
                if temp in thirty:
                    ans+=1
                # print('valid')
                    break
                else:
                    nok+=1
                    break
            if not darkmode:
                if temp in forty:
                    continue
                if temp in thirty:
                    print('darkmode')
                    darkmode = True
                    nextforty = True
                    continue
            elif darkmode:
                print('darkmode')
                if temp in forty and nextforty and i != cut -2:
                    nextforty = False
                    continue
                elif temp in thirty and not nextforty:
                    nextforty = True
                    continue
                else:
                    print('nok')
                    nok+=1
                    break
            else:
                # nok+=1
                break
ans

darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode

243

In [153]:
ans

243

In [147]:
267+89

356

In [66]:
aoc.flatten([['aa'],['bb'],[['cc','dd']]])

['aa', 'bb', 'cc', 'dd']

In [67]:
def is_single_list(item):
    if isinstance(item, list):
        for i in item:
            if isinstance(i, list):
                return False
        else:
            return True
    else:
        return True




In [68]:
'2' in {'22'}

False

In [69]:
def getoptions(key):
    ins = rules[key]
    alloptions = []
    for option in ins: # go through all the options, can be 1 if 1 option. this can still yield multiple results through recursion
        # print('option', option)
        ans = []
        for i in option:
            if i.isnumeric():
                res = getoptions(i)
                ans.append(res)
            else:
                ans.append(i) # i is a or b

        temp = [''.join(comb) for comb in itertools.product(*ans)]
            # if len(temp)==1:
            #     alloptions.append(temp[0])
            # else:
        alloptions.append(temp)
    if len(alloptions)==1:
        return alloptions[0]
    else:
        # print(aoc.flatten(alloptions))
        return aoc.flatten(alloptions)
# getoptions('0')